In [2]:
import numpy as np
import glob
import pyproj
import xarray as xr
from scipy.interpolate import griddata
from matplotlib.dates import date2num
import datetime

### Testing tracks and end criteria


Test run from M2. Trying with and without the "no ice" criteria.

In [3]:
print('Running other notebooks: Preparing data..\r', end = '')
%run "Load and prepare data.ipynb"
print('Running other notebooks: Defining particle tracking class..', end = '')
%run "Define particle tracking class.ipynb"
print('  ..done.')

Running other notebooks: Defining particle tracking class..  ..done.


In [4]:
coords =pickle_load('/home/oyvindl/work/arven_etter_nansen/data/aen_coords_v2.p')
moor = 'm2'
lon = coords[moor].lon
lat = 73#coords[moor].lat
time0 = datetime.date(2019, 3, 15)

NameError: name 'pickle_load' is not defined

In [ ]:
input_dict = {'D':D, 'lat0':lat, 'lon0':lon, 'time0':time0, 'MASK':MASK, 
              'interp_method':'linear', 'end_criteria':'ABC' }

In [ ]:
P = ice_particle(input_dict)
P.backtrack()

In [ ]:
P.end_reason

In [215]:
# Same but without "no ice" (A) end criterion

input_dict2 = input_dict.copy()
input_dict2['end_criteria'] = 'BC'
P2 = ice_particle(input_dict2)
print('End criteria: %s'%P2.end_criteria)
P2.backtrack()

Setting up object ..
 ..done.
End criteria: BC


In [126]:
from oyv.map import quickmap

In [140]:
ca()

In [216]:
m = quickmap.map_stere(15, 15, oc_col = 'w')
m.pcolormesh(D.longitude.values, D.latitude.values, D.u.sel(time = P.date_origin), latlon = True)
m.plot(P.lon, P.lat, 'k', latlon = True)
m.plot(P.lon0, P.lat0, '*r', latlon = True)
m.plot(P.lon0, P.lat0, '*y', latlon = True)
m.plot(P2.lon, P2.lat, 'y', latlon = True)
m.plot(P2.lon0, P2.lat0, '*r', latlon = True)
m.plot(P2.lon0, P2.lat0, '*y', latlon = True)

KeyError: "not all values found in index 'time'. Try setting the `method` keyword argument (example: method='nearest')."

#### STATUS/ NOTE

- Do more extensive testing.
- Do I need all the 4 coordinate transforms?
- Check what "linear" in "griddata" actually does.. (**X**)
- Check what to do if we initially have open water

In [155]:
fig, ax = plt.subplots()
ax.plot(P.X_ocean_flat, P.Y_ocean_flat, '.')

### Quick sanity check of "griddata"
(Note: I'm faily sure this one is good from my earlier checks..)


**Note**: "Linear" does *not* do bilinear interpolation (see this: https://stackoverflow.com/questions/49185127/unexpected-results-when-using-scipy-interpolate-griddata).

This is the explanation ofrom the docs:

      ``linear``
      tessellate the input point set to N-D
      simplices, and interpolate linearly on each simplex. See
      `LinearNDInterpolator` for more details.

Could call it *"2D linear interpolation"*, I think..

In [180]:
data.min()

-0.5002097168386255

In [176]:
x_data = [0, 0, 1, 2, 2]
y_data = [0, 2, 1, 0, 1]
data = np.random.randn(5)

In [190]:

x_ip = np.linspace(0, 2, 20)
y_ip = np.linspace(0, 2, 20)
X_ip, Y_ip = np.meshgrid(x_ip, y_ip)

In [191]:
data1 = griddata((x_data, y_data), data, (X_ip, Y_ip), 
                                 method = 'nearest')
data2 = griddata((x_data, y_data), data, (X_ip, Y_ip), 
                                 method = 'linear')

In [195]:
fig, ax = plt.subplots(1, 2, sharex = True, sharey = True)
cmap = col.cm('balance')
vmin = -1
vmax = 1

ax[0].pcolormesh(x_ip, y_ip, data1, cmap = cmap, vmin = vmin, vmax = vmax,)
ax[1].pcolormesh(x_ip, y_ip, data2, cmap = cmap, vmin = vmin, vmax = vmax,)

ax[0].set_title('NEAREST')
ax[1].set_title('LINEAR')

for axn in ax:
    axn.scatter(x_data, y_data, 40, data, cmap = cmap, vmin = vmin, vmax = vmax, ec = 'k')
    axn.set_aspect('equal')

In [228]:
pwd

'/home/oyvindl/work/code/git_public/sea_ice_track/code'